# 目的
spaCyのdisplaCyを用いて、文章とタグを可視化する

In [2]:
import polars as pl

import spacy
from spacy.tokens import Span
from spacy import displacy

In [3]:
# 一度だけ必要
#!poetry run python -m spacy download en_core_web_sm

In [4]:
# https://matplotlib.org/2.0.2/examples/color/named_colors.html
options = {
    "colors": {
        "B-NAME_STUDENT": "aqua",
        "I-NAME_STUDENT": "aliceblue",
        "B-EMAIL": "forestgreen",
        "I-EMAIL": "greenyellow",
        "B-USERNAME": "hotpink",
        "I-USERNAME": "lightpink",
        "B-ID_NUM": "purple",
        "I-ID_NUM": "rebeccapurple",
        "B-PHONE_NUM": "red",
        "I-PHONE_NUM": "salmon",
        "B-URL_PERSONAL": "slategrey",
        "I-URL_PERSONAL": "silver",
        "B-STREET_ADDRESS": "brown",
        "I-STREET_ADDRESS": "chocolate",
    }
}

In [5]:
train = pl.read_json("../data/train.json")

In [6]:
documents = train.get_column("document").to_list()
tokens = train.get_column("tokens").to_list()
# full_texts = pii_dataset_fixed.get_column("text").to_list()
labels = train.get_column("labels").to_list()

In [7]:
# class PllLabelRenderer:
#     def __init__(
#         self,
#         documents: list[str],
#         full_texts: list[str],
#         labels: list[str],
#         options: dict[str, dict[str, str]],
#     ) -> None:
#         self.nlp = spacy.blank("en")
#         self.documents = documents
#         self.full_texts = full_texts
#         self.labels = labels
#         self.options = options

#     def render(
#         self, render_idx: int | None = None, documents_id: int | None = None
#     ) -> None:
#         if render_idx and documents_id:
#             raise Exception("Don't Use Both render_idx and documents_id")
#         if documents_id:
#             render_idx = self.documents.index(documents_id)
#         doc = self.nlp(self.full_texts[render_idx])
#         doc.ents = [
#             Span(doc, idx, idx + 1, label=label)
#             for idx, label in enumerate(self.labels[render_idx])
#             if label != "O"
#         ]
#         displacy.render(doc, style="ent", jupyter=True, options=options)

In [8]:
import spacy
from spacy.tokens import Span, Doc
from spacy import displacy

# https://matplotlib.org/2.0.2/examples/color/named_colors.html
options = {
    "colors": {
        "B-NAME_STUDENT": "aqua",
        "I-NAME_STUDENT": "aliceblue",
        "B-EMAIL": "forestgreen",
        "I-EMAIL": "greenyellow",
        "B-USERNAME": "hotpink",
        "I-USERNAME": "lightpink",
        "B-ID_NUM": "purple",
        "I-ID_NUM": "rebeccapurple",
        "B-PHONE_NUM": "red",
        "I-PHONE_NUM": "salmon",
        "B-URL_PERSONAL": "slategrey",
        "I-URL_PERSONAL": "silver",
        "B-STREET_ADDRESS": "brown",
        "I-STREET_ADDRESS": "chocolate",
    }
}


class PiiLabelRenderer:
    def __init__(
        self,
        documents: list[str],
        tokens: list[str],
        labels: list[str],
        options: dict[str, dict[str, str]],
    ) -> None:
        self.documents = documents
        self.tokens = tokens
        self.labels = labels
        self.options = options
        self.nlp = spacy.blank("en")

    def get_original_text(
        self, render_idx: int | None = None, documents_id: int | None = None
    ) -> str:
        if render_idx and documents_id:
            raise Exception("Don't Use Both render_idx and documents_id")
        if documents_id:
            render_idx = self.documents.index(documents_id)

        return " ".join(self.tokens[render_idx])

    def render(
        self, render_idx: int | None = None, documents_id: int | None = None
    ) -> None:
        if render_idx and documents_id:
            raise Exception("Don't Use Both render_idx and documents_id")
        if documents_id:
            render_idx = self.documents.index(documents_id)

        doc = Doc(self.nlp.vocab, words=self.tokens[render_idx])
        # エンティティをDocに追加
        ents = []
        label_seq: list[str] = self.labels[render_idx]
        for i, label in enumerate(label_seq):
            if label == "O":
                continue
            ents.append(Span(doc, start=i, end=i + 1, label=label))

        doc.ents = ents

        displacy.render(doc, style="ent", jupyter=True, options=options)

In [9]:
PLR = PiiLabelRenderer(
    documents=documents, tokens=tokens, labels=labels, options=options
)

In [10]:
PLR.render(documents_id=330)